In [1]:
!pip install -qU open_clip_torch transformers

In [2]:
!pip install -qU pinecone-client

In [3]:
!pip install -qU torch torchvision requests lxml tqdm 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.1.2 requires requests<2.29,>=2.24.0, but you have requests 2.30.0 which is incompatible.
ydata-profiling 4.1.2 requires tqdm<4.65,>=4.48.2, but you have tqdm 4.65.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.4.0 which is incompatible.
beatrix-jupyterlab 2023.46.184821 requires jupyter-server~=1.16, but you have jupyter-server 2.5.0 which is incompatible.
apache-

In [4]:
import open_clip
import torch
from transformers import MarianMTModel, MarianTokenizer
from PIL import Image
import pinecone
import os
import requests
from io import BytesIO
from lxml import etree
#import spacy
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
model_name = 'Helsinki-NLP/opus-mt-ru-en'
tr_tokenizer = MarianTokenizer.from_pretrained(model_name)#.to(device)
tr_model = MarianMTModel.from_pretrained(model_name).to(device)  # Move the model to GPU if available


/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
# Create a function for translation
def translate(text: str, model, tokenizer):
    # Tokenize the source text
    tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt").to(device)  # Move the tokens to GPU if available

    # Perform the translation
    translated = model.generate(**tokenized_text)

    # Decode the translation
    translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    return translated_text[0]

# Example usage
russian_text = "Стеллаж ПК-9, 6 шт. вариант 1"
english_translation = translate(russian_text, tr_model, tr_tokenizer)
print(english_translation)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3732: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


PK-9 stellage, 6 unit version 1


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
im_model, _, im_transform = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)
im_model.cuda().eval()

CoCa(
  (text): TextTransformer(
    (token_embedding): Embedding(49408, 768)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_final): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (visual): VisionTransformer(
    (patchnorm_pre_ln): Identity()
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(1

In [10]:
def get_caption(image, model, transform):
    im = transform(image).unsqueeze(0).to(device)#cuda()
    with torch.no_grad():
        if device == "cuda":
            with torch.cuda.amp.autocast():
                generated = model.generate(im)
        else:
            generated = model.generate(im)
    caption = open_clip.decode(generated[0])#.split("")[0].replace("", "")[:-2]
    return caption

In [13]:
path_name="2.png"
im = Image.open(path_name).convert("RGB")
caption = get_caption(im, im_model, im_transform)
print(caption)

<start_of_text>a dresser that is sitting on the floor . <end_of_text>


In [12]:
# Authenticate with your Pinecone account
api_key = <API KEY>  # Replace with your actual API key
pinecone.init(api_key, environment=<GCP>)# Replace with your actual GVP 


# Create a new index (vector database) in Pinecone
index_name = "clip-embeddings"

if index_name not in pinecone.list_indexes():
    # create the index
    pinecone.create_index(
      index_name,
      dimension=1536,#1024, #512
      metric="cosine",
      pod_type="s1"
    )

# Instantiate connection to your Pinecone index
index = pinecone.Index(index_name)

In [26]:
#Download and parse the XML feed
url = <FEED># Replace with your actual feed 
response = requests.get(url)
xml_content = response.content
root = etree.fromstring(xml_content)
# Save to file
with open('response.xml', 'wb') as file:
    file.write(xml_content)

In [13]:
from open_clip import tokenizer
#Download and parse the XML feed
url = <FEED> # Replace with your actual feed 
response = requests.get(url)
xml_content = response.content
root = etree.fromstring(xml_content)
print('loaded')
    
# Extract required fields and download images
offers = root.xpath("//offers/offer")
BATCH_SIZE = 100

#offers=offers[59500:]

# Iterate through batches 
from tqdm.auto import tqdm  # Use auto to ensure compatibility with different environments
with tqdm(range(0, len(offers), BATCH_SIZE), desc="Processing batches") as batch_progress:
    for batch_start in batch_progress:
        batch_end = min(batch_start + BATCH_SIZE, len(offers))
        batch_offers = offers[batch_start:batch_end]

        image_binaries = []
        image_urls = []
        product_urls = []
        image_names = []
        image_descriptions = []
        image_names_en = []
        image_autocapts_en = []

        with tqdm(batch_offers, desc="Processing offers", leave=False) as offer_progress:  # leave=False to clear the progress bar after completion
            for offer in offer_progress:
                try:
                    name = offer.find("name").text
                    product_url = offer.find("url").text
                    description = offer.find("description").text
                    picture_url = offer.find("picture").text

                    image_response = requests.get(picture_url)
                    image = Image.open(BytesIO(image_response.content))

                    image_names.append(name)
                    
                    image_descriptions.append(description)
                    image_binaries.append(image)
                    image_urls.append(picture_url)
                    product_urls.append(product_url)
                    
                                       
                    image_name_en = translate(name, tr_model, tr_tokenizer)
                    image_names_en.append(image_name_en)
                    
                    image_autocapt_en = get_caption(image, im_model, im_transform)
                    # Remove the tags
                    image_autocapt_en = (image_autocapt_en.replace("<start_of_text>", "").replace("<end_of_text>", "")).strip()
                    # Remove leading and trailing whitespaces
                    #acapt = acapt.strip()
                    #image_autocapt_en = acapt
                    image_autocapts_en.append(image_autocapt_en)
                    
                    
                    #print("name_ru:", name)
                    #print("name_en:", image_name_en)
                    #print("autocapt_en:", image_autocapt_en)
                    #print("picture_url:", picture_url)
                    #print("_____________")
                except:
                    print('processing error')

        # Preprocess images and compute embeddings
        preprocessed_images = torch.stack([im_transform(image) for image in image_binaries]).to(device)
        text_inputs = tokenizer.tokenize(image_autocapts_en).to(device)   
        
        with torch.no_grad():
             image_embeddings = im_model.encode_image(preprocessed_images)
             text_embeddings = im_model.encode_text(text_inputs)

        combined_embeddings = torch.cat((image_embeddings, text_embeddings), dim=1)

        # Upsert the image embeddings and product names into Pinecone
        upserts = []
        for name, name_en, autoname_en, embedding, image_url, image_descr, product_url in zip(image_names, image_names_en, image_autocapts_en, combined_embeddings, image_urls, image_descriptions, product_urls):
            idx_s = str(hash(image_url))
            image_descr = image_descr if image_descr is not None else ""  # Set image_descr to an empty string if it is None
            #print(product_url)
            upserts.append({
                "id": idx_s,
                "values": embedding.tolist(),
                "metadata": {"name_ru": name, "name_en": name_en, "autoname_en":autoname_en, "image_url": image_url, 'image_descr': image_descr, 'product_url':product_url},
            })

        #Upsert the data into Pinecone
        index.upsert(upserts)

loaded


Processing batches:   0%|          | 0/191 [00:00<?, ?it/s]

Processing offers:   0%|          | 0/100 [00:00<?, ?it/s]

Processing offers:   0%|          | 0/100 [00:00<?, ?it/s]

Processing offers:   0%|          | 0/100 [00:00<?, ?it/s]

...

Processing offers:   0%|          | 0/100 [00:00<?, ?it/s]

Processing offers:   0%|          | 0/100 [00:00<?, ?it/s]

Processing offers:   0%|          | 0/100 [00:00<?, ?it/s]

processing error


Processing offers:   0%|          | 0/74 [00:00<?, ?it/s]

In [ ]:
images=[]
images.append(im)
#preprocessed_images = torch.stack([preprocess(image) for image in images]).to(device)

with torch.no_grad():
          image_embeddings = model.encode_image(im)
        
print(image_embeddings)
#print(len(image_embeddings[0]))

In [ ]:
import numpy as np

image_input = torch.tensor(torch.stack(images))
with torch.no_grad():
    image_features = model.encode_image(im).float()
print(image_features)